# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
from unityagents import UnityEnvironment
from ddpg_agent import Agent

%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
# Load Reacher env with 20 agents
env = UnityEnvironment(
    file_name='Reacher_Linux_20/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 5. Instantiate DDPG Agent

In [6]:
random_seed = 7
train_mode = True
agent = Agent(state_size=state_size, action_size=action_size, random_seed=random_seed)

### 6. DDPG function

In [7]:
def ddpg(n_episodes=2000, max_t=1000, print_every=10, learn_every=20, num_learn=10, goal_score=30, ):
    total_scores_deque = deque(maxlen=100)
    total_scores = []

    for i_episode in range(1, n_episodes + 1):
        # reset environment
        env_info = env.reset(train_mode=train_mode)[brain_name]
        # get current state for each agent
        states = env_info.vector_observations
        # initialize the score (for each agent)
        scores = np.zeros(num_agents)
        agent.reset()
        start_time = time.time()

        for t in range(max_t):
            # select an action
            actions = agent.act(states, add_noise=True)
            # send actions to environment
            env_info = env.step(actions)[brain_name]
            # get next state
            next_states = env_info.vector_observations
            # get reward
            rewards = env_info.rewards
            # check if episode has finished
            dones = env_info.local_done

            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                # send actions to the agent
                agent.step(state, action, reward, next_state, done)
            # pass states to next time step
            states = next_states
            # update the scores
            scores += rewards

            if t % learn_every == 0:
                for _ in range(num_learn):
                    agent.start_learn()
            # exit loop if episode finished
            if np.any(dones):
                break

        duration = time.time() - start_time
        # save lowest score for a single agent
        min_score = np.min(scores)
        # save highest score for a single agent
        max_score = np.max(scores)
        # save mean score for the episode
        mean_score = np.mean(scores)
        total_scores_deque.append(mean_score)
        total_scores.append(mean_score)
        total_average_score = np.mean(total_scores_deque)

        print('\rEpisode {}\tTotal Average Score: {:.2f}\tMean: {:.2f}\tMin: {:.2f}\tMax: {:.2f}\tDuration: {:.2f}'
              .format(i_episode, total_average_score, mean_score, min_score, max_score, duration))

        if i_episode % print_every == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print(
                '\rEpisode {} ({} sec)  -- \tMin: {:.1f}\tMax: {:.1f}\tMean: {:.1f}'.format(i_episode, round(duration),
                                                                                            min_score, max_score,
                                                                                            mean_score))

        if total_average_score >= goal_score and i_episode >= 100:
            print(
                'Problem Solved after {} epsisodes Total Average score: {:.2f}'.format(i_episode, total_average_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break

    return total_scores

### 7. Start Training

In [8]:
scores = ddpg()

/home/nikolaos/PycharmProjects/Continuous Control/ddpg_agent.py:118: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)


Episode 1	Total Average Score: 0.28	Mean: 0.28	Min: 0.00	Max: 0.66	Duration: 7.71
Episode 2	Total Average Score: 0.38	Mean: 0.48	Min: 0.00	Max: 1.83	Duration: 8.26
Episode 3	Total Average Score: 0.57	Mean: 0.95	Min: 0.17	Max: 1.66	Duration: 8.79
Episode 4	Total Average Score: 0.69	Mean: 1.05	Min: 0.12	Max: 2.14	Duration: 8.88
Episode 5	Total Average Score: 0.82	Mean: 1.35	Min: 0.46	Max: 3.35	Duration: 9.05
Episode 6	Total Average Score: 0.86	Mean: 1.05	Min: 0.09	Max: 2.22	Duration: 9.30
Episode 7	Total Average Score: 0.91	Mean: 1.22	Min: 0.35	Max: 2.52	Duration: 9.48
Episode 8	Total Average Score: 0.98	Mean: 1.44	Min: 0.26	Max: 2.82	Duration: 9.87
Episode 9	Total Average Score: 1.01	Mean: 1.24	Min: 0.25	Max: 2.73	Duration: 10.15
Episode 10	Total Average Score: 1.05	Mean: 1.44	Min: 0.11	Max: 2.61	Duration: 10.61
Episode 10 (11 sec)  -- 	Min: 0.1	Max: 2.6	Mean: 1.4
Episode 11	Total Average Score: 1.10	Mean: 1.61	Min: 0.29	Max: 3.11	Duration: 11.11
Episode 12	Total Average Score: 1.14	Mea

Episode 91	Total Average Score: 24.88	Mean: 38.76	Min: 37.09	Max: 39.44	Duration: 31.66
Episode 92	Total Average Score: 25.02	Mean: 37.65	Min: 33.80	Max: 39.40	Duration: 31.17
Episode 93	Total Average Score: 25.15	Mean: 37.97	Min: 35.20	Max: 39.36	Duration: 33.15
Episode 94	Total Average Score: 25.29	Mean: 37.76	Min: 34.99	Max: 39.16	Duration: 30.34
Episode 95	Total Average Score: 25.42	Mean: 37.93	Min: 36.47	Max: 39.43	Duration: 31.94
Episode 96	Total Average Score: 25.56	Mean: 38.69	Min: 36.51	Max: 39.57	Duration: 33.97
Episode 97	Total Average Score: 25.69	Mean: 37.69	Min: 34.89	Max: 39.13	Duration: 31.91
Episode 98	Total Average Score: 25.81	Mean: 38.39	Min: 34.78	Max: 39.45	Duration: 31.25
Episode 99	Total Average Score: 25.94	Mean: 38.39	Min: 35.41	Max: 39.25	Duration: 30.84
Episode 100	Total Average Score: 26.06	Mean: 38.12	Min: 35.22	Max: 39.60	Duration: 31.63
Episode 100 (32 sec)  -- 	Min: 35.2	Max: 39.6	Mean: 38.1
Episode 101	Total Average Score: 26.45	Mean: 38.50	Min: 36.30	

### 8. Plot the scores

In [9]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(np.arange(1, len(scores) + 1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 9. Close Unity Environment 

In [10]:
env.close()